In [9]:
import os
%pwd

'c:\\Users\\alok4\\Desktop\\Text-Summarizer\\research'

In [10]:
os.chdir("..")

In [11]:
%pwd

'c:\\Users\\alok4\\Desktop\\Text-Summarizer'

In [12]:
from pathlib import Path

In [23]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Defaulting to user installation because normal site-packages is not writeable
Found existing installation: transformers 4.55.4
Uninstalling transformers-4.55.4:
  Successfully uninstalled transformers-4.55.4
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.55.4-py3-none-any.whl.metadata (41 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.55.4-py3-none-any.whl (11.3 MB)
Using cached accelerate-1.10.1-py3-none-any.whl (374 kB)

   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ----------------------------

In [24]:
from dataclasses import dataclass
@dataclass
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int 
    save_steps: float
    gradient_accumulation_steps: int



In [25]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [26]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        create_directories([self.config.artifact_root],verbose=True)
    def get_model_trainer_config(self)->ModelTrainingConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        create_directories([config.root_dir],verbose=True)
        model_training_config=ModelTrainingConfig(root_dir=config.root_dir,data_path=config.data_path,
                                                  model_ckpt=config.model_ckpt,num_train_epochs=params.num_train_epochs,warmup_steps=params.warmup_steps,per_device_train_batch_size=params.per_device_train_batch_size,weight_decay=params.weight_decay,logging_steps=params.logging_steps,evaluation_strategy=params.evaluation_strategy,eval_steps=params.eval_steps,save_steps=params.save_steps,gradient_accumulation_steps=params.gradient_accumulation_steps)
        return model_training_config
        

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Training
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_from_disk


In [28]:
class ModelTrainer:
    def __init__(self,config:ModelTrainingConfig):
        self.config=config
    def train(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        print(device)
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)
        
        dataset_samsum_pt=load_from_disk(self.config.data_path)
        trainer_args=TrainingArguments(
            output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,
            per_device_train_batch_size=1,per_device_eval_batch_size=1,
            weight_decay=0.01,logging_steps=10,
            eval_strategy='steps',eval_steps=500,save_steps=1e6,
            gradient_accumulation_steps=16
        )
        trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt['test'],eval_dataset=dataset_samsum_pt['validation'])
        trainer.train()

        ## save model
        model_pegasus.save_pretrained("pegasus-samsum-model")
        tokenizer.save_pretrained("tokenizer")

In [29]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer=ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-08-29 00:49:25,207:INFO:common:Yaml file:config\config.yaml loaded successfully]
[2025-08-29 00:49:25,208:INFO:common:Yaml file:params.yaml loaded successfully]
[2025-08-29 00:49:25,209:INFO:common:Created Directory at: artifacts]
[2025-08-29 00:49:25,210:INFO:common:Created Directory at: artifacts/model_trainer]
cpu


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


[2025-08-29 00:49:27,027:WARNING:file_download:Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`]


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


[2025-08-29 01:14:49,197:WARNING:file_download:Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: Directory artifacts/data_transformation/samsum_dataset not found